# Section 5: Models

TODO

## Classifiers

TODO

### KNN

TODO

## Detectors

TODO

## Clustering

TODO

## Libraries

TODO

In [1]:
import pandas as pd
# dataset location
data_path = "./datasets/brazilian-malware.csv"
# read CSV dataset
data = pd.read_csv(data_path, keep_default_na=False)

TODO

In [2]:
# numerical attributes
NUMERICAL_ATTRIBUTES = ['BaseOfCode', 'BaseOfData', 'Characteristics', 'DllCharacteristics', 
                      'Entropy', 'FileAlignment', 'ImageBase', 'Machine', 'Magic',
                      'NumberOfRvaAndSizes', 'NumberOfSections', 'NumberOfSymbols', 'PE_TYPE',
                      'PointerToSymbolTable', 'Size', 'SizeOfCode', 'SizeOfHeaders',
                      'SizeOfImage', 'SizeOfInitializedData', 'SizeOfOptionalHeader',
                      'SizeOfUninitializedData']

# textual attributes
TEXTUAL_ATTRIBUTES = ['Identify', 'ImportedDlls', 'ImportedSymbols']

# label used to classify
LABEL = 'Label'

# attributes that are not used
UNUSED_ATTRIBUTES = ['FirstSeenDate', 'SHA1', 'TimeDateStamp']

TODO

In [3]:
label = data[LABEL].values
# remove unused attributes and label
for a in UNUSED_ATTRIBUTES:
    del data[a]
del data[LABEL]

TODO

In [4]:
# split data in half
def split_data(data):
    # get mid of data
    mid = int((len(data) + 1)/2)
    # split data into train and test
    train_data = data[:mid]
    test_data = data[mid:]
    # return train and test data
    return(train_data, test_data)

In [5]:
data, _ = split_data(data)
label, _ = split_data(label)
train_data, test_data = split_data(data)
train_label, test_label = split_data(label)

TODO

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
# extract features from textual attributes
def textual_feature_extraction(train_data, test_data, extractor=TfidfVectorizer(max_features=100)):
    vectorizer = extractor
    # train vectorizer
    vectorizer.fit(train_data)
    # transform train and test data to features
    train_features = vectorizer.transform(train_data)
    test_features = vectorizer.transform(test_data)
    # return train and test features
    return(train_features, test_features)

Obtain numerical attributes:

In [7]:
train_features = train_data[NUMERICAL_ATTRIBUTES].values
test_features = test_data[NUMERICAL_ATTRIBUTES].values

In [8]:
train_features.shape, test_features.shape

((12546, 21), (12545, 21))

Obtain textual attributes and append to features array:

In [9]:
import numpy as np
# extract features from each textual attribute
for a in TEXTUAL_ATTRIBUTES:
    # extract features from current attribute
    train_texts, test_texts = textual_feature_extraction(train_data[a], test_data[a])
    train_features = np.concatenate((train_features, train_texts.toarray()), axis=1)
    test_features = np.concatenate((test_features, test_texts.toarray()), axis=1)

In [10]:
train_features.shape, test_features.shape

((12546, 321), (12545, 321))

Normalization:

In [11]:
from sklearn.preprocessing import MinMaxScaler
def normalization(train_features, test_features, scaler=MinMaxScaler()):
    # train minmax
    scaler.fit(train_features)
    # transform features
    train_features_norm = scaler.transform(train_features)
    test_features_norm = scaler.transform(test_features)
    # return normalized train and test features
    return(train_features_norm, test_features_norm)

In [12]:
train_features_norm, test_features_norm = normalization(train_features, test_features)

### Scikit-Learn

TODO

#### KNN

TODO

In [13]:
from sklearn.neighbors import KNeighborsClassifier
# initialize classifier
clf = KNeighborsClassifier(n_neighbors=3)
# train classifier
clf.fit(train_features_norm, train_label)
# predict test classes
test_pred = clf.predict(test_features_norm)
# print test pred and real labels shape
print(test_pred.shape, test_label.shape)

(12545,) (12545,)


#### Random Forest

TODO

In [14]:
from sklearn.ensemble import RandomForestClassifier
# initialize classifier
clf = RandomForestClassifier(n_estimators=10)
# train classifier
clf.fit(train_features_norm, train_label)
# predict test classes
test_pred = clf.predict(test_features_norm)
# print test pred and real labels shape
print(test_pred.shape, test_label.shape)

(12545,) (12545,)


#### SVM

TODO

In [15]:
from sklearn.svm import SVC
# initialize classifier
clf = SVC(kernel="linear")
# train classifier
clf.fit(train_features_norm, train_label)
# predict test classes
test_pred = clf.predict(test_features_norm)
# print test pred and real labels shape
print(test_pred.shape, test_label.shape)

(12545,) (12545,)


#### K-Means

TODO

In [16]:
from sklearn.cluster import KMeans
# initialize kmeans
clustering = KMeans(n_clusters=2)
# fit kmeans (note that we do not need to use labels here)
# and predict train classes using the clusters created
train_pred = clustering.fit_predict(train_features_norm)
# print train pred and real labels shape
print(train_pred.shape, train_label.shape)

(12546,) (12546,)


#### DBScan

TODO

### Scikit-Multiflow

TODO

#### KNN

TODO

In [31]:
from skmultiflow.meta import AdaptiveRandomForest
from skmultiflow.data import DataStream
# initialize classifier
clf = AdaptiveRandomForest(n_estimators=10, disable_weighted_vote=True) # disable_weighted_vote=False produces a bug
# fit classifier
clf.partial_fit(train_features_norm, train_label)

AdaptiveRandomForest(binary_split=False, disable_weighted_vote=True,
                     drift_detection_method=ADWIN(delta=0.001), grace_period=50,
                     lambda_value=6, leaf_prediction='nba',
                     max_byte_size=33554432, max_features=18,
                     memory_estimate_period=2000000, n_estimators=10,
                     nb_threshold=0, no_preprune=False, nominal_attributes=None,
                     performance_metric='acc', random_state=None,
                     remove_poor_atts=False, split_confidence=0.01,
                     split_criterion='info_gain', stop_mem_management=False,
                     tie_threshold=0.05,
                     warning_detection_method=ADWIN(delta=0.01))

In [32]:
# create a stream with test features
stream = DataStream(test_features_norm, test_label)
# prepare stream for use
stream.prepare_for_use()
# create prediction array
test_pred = []
# iterate over stream
while stream.has_more_samples():
    # get next sample features and label from stream
    sample_features, sample_label = stream.next_sample(1)
    # predict sample
    sample_pred = clf.predict(sample_features)
    # add predicted labels to test_pred
    for l in sample_pred:
        test_pred.append(l)
    # update model with new sample
    clf.partial_fit(sample_features, sample_label)
# turn test_pred into numpy array
test_pred = np.array(test_pred)
print(test_pred.shape, test_label.shape)

(12545,) (12545,)


#### Random Forest

TODO

#### SVM

TODO

### Keras

TODO

#### Multi-Layer Perceptron

TODO

In [40]:
import keras
from keras.models import Sequential
from keras.layers import Dense
# converts labels to a categorical one-hot-vector
train_label_onehot = keras.utils.to_categorical(train_label, num_classes=2)
test_label_onehot = keras.utils.to_categorical(test_label, num_classes=2)
# initialize sequential network
model = Sequential()
# add fully-connected hidden layer with 200 units
model.add(Dense(200, activation='relu', input_dim=train_features_norm.shape[1]))
# add fully-connected hidden layer with 100 units
model.add(Dense(100, activation='relu'))
# output layer
model.add(Dense(2, activation='softmax'))
# compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# fit model with data
model.fit(train_features_norm, train_label_onehot, validation_split=0.33, epochs=10, batch_size=128)
# predict classes
test_pred = model.predict_classes(test_features_norm)
print(test_pred.shape, test_label.shape)

Train on 8405 samples, validate on 4141 samples
Epoch 1/10
8405/8405 [==============================] - 1s 95us/step - loss: 0.0219 - acc: 0.9996 - val_loss: 5.3590 - val_acc: 0.5576
Epoch 2/10
8405/8405 [==============================] - 0s 40us/step - loss: 3.4011e-05 - acc: 1.0000 - val_loss: 6.6105 - val_acc: 0.5576
Epoch 3/10
8405/8405 [==============================] - 0s 35us/step - loss: 1.0791e-06 - acc: 1.0000 - val_loss: 6.9370 - val_acc: 0.5576
Epoch 4/10
8405/8405 [==============================] - 0s 34us/step - loss: 1.6115e-07 - acc: 1.0000 - val_loss: 7.0641 - val_acc: 0.5576
Epoch 5/10
8405/8405 [==============================] - 0s 35us/step - loss: 1.1825e-07 - acc: 1.0000 - val_loss: 7.0764 - val_acc: 0.5576
Epoch 6/10
8405/8405 [==============================] - 0s 37us/step - loss: 1.1381e-07 - acc: 1.0000 - val_loss: 7.0839 - val_acc: 0.5576
Epoch 7/10
8405/8405 [==============================] - 0s 33us/step - loss: 1.1218e-07 - acc: 1.0000 - val_loss: 7.0877 -